## III - Неопределенность оптимального портфеля. Оптимизация CVaR. 

Целью работы является оценка неопределенности оптимального портфеля для нормального многомерного распределения и двух способов вычисления оптимального портфеля (оптимизация в модели Марковица и оптимизация CVaR)

### Подготовка модели

#### Загружаем тикеры с фоднового рынка NASDAQ

In [27]:
import pandas as pd
import numpy as np

DATA_PATH    = './downloader-data'
TICKERS_PATH = './tickers'

def read_tickers(stock_markets) -> pd.DataFrame:
    '''Returns pandas dataframe containing all tickers for the @stock_markets '''
    ticker_files = [f'{TICKERS_PATH}/{sm}.csv' for sm in stock_markets]
    tickers = pd.concat([pd.read_csv(tf) for tf in ticker_files], ignore_index=True)
    return tickers


stock_markets = ['NASDAQ']
tickers = read_tickers(stock_markets)
tickers

,ticker,company
0,AAIT,iShares MSCI All Country Asia Information Tech...
1,AAL,"American Airlines Group, Inc."
2,AAME,Atlantic American Corporation
3,AAOI,"Applied Optoelectronics, Inc."
4,AAON,"AAON, Inc."
...,...,...
2962,ZN,Zion Oil & Gas Inc
2963,ZNGA,Zynga Inc.
2964,ZSPH,"ZS Pharma, Inc."
2965,ZU,"zulily, inc."


#### Загружаем исторические данные за 2021 год для полученных тикеров

In [28]:
def read_historical_data(tickers):
    '''Returns dict {ticker : historical data for the ticker}'''
    data_for_ticker = {}
    for index, (ticker, name) in tickers.iterrows():
        try:
            data = pd.read_csv(f'{DATA_PATH}/{ticker}.csv')
            if len(data) > 100:
                data_for_ticker[ticker] = data
        except:
            pass
    return data_for_ticker


data_for_ticker = read_historical_data(tickers)

#### Пример данных для AAPL:

In [29]:
data_for_ticker['AAPL']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-11-30,116.597423,120.584682,116.437929,118.670799,169410200,0.0,0
1,2020-12-01,120.624557,123.076720,119.627742,122.329109,127728200,0.0,0
2,2020-12-02,121.631329,122.977035,120.504931,122.687958,89004200,0.0,0
3,2020-12-03,123.126555,123.385729,121.820730,122.548409,78967600,0.0,0
4,2020-12-04,122.209495,122.468669,121.132933,121.860611,78260400,0.0,0
...,...,...,...,...,...,...,...,...
248,2021-11-23,161.119995,161.800003,159.059998,161.410004,96041900,0.0,0
249,2021-11-24,160.750000,162.139999,159.639999,161.940002,69463600,0.0,0
250,2021-11-26,159.570007,160.449997,156.360001,156.809998,76959800,0.0,0
251,2021-11-29,159.369995,161.190002,158.789993,160.240005,88748200,0.0,0


#### Выбираем 20 активов

In [30]:
assets = ['MDLZ', 'MSFT', 'NXPI', 'PCAR', 'INTC',
          'NVDA', 'ILMN', 'DXCM', 'ROST', 'LULU',
          'PRFT', 'FTNT', 'WIRE', 'NFLX', 'GAIN',
          'SYNA', 'NYMTP', 'ANAT', 'TRNS', 'SFBS']

assets_data = {}
for asset in assets:
    assets_data[asset] = data_for_ticker[asset]

#### Оцениваем математические ожидания доходностей

In [31]:
import pandas as pd
import numpy as np

def add_logret(ticker_data: pd.DataFrame, by_column='Close') -> pd.DataFrame:
    '''Returns @ticker_data with calculated "logret" and "-logret" columns'''
    ticker_data = ticker_data.assign(logret=np.log(ticker_data[by_column]).diff())
    ticker_data['-logret'] = ticker_data['logret'].mul(-1)
    return ticker_data


def get_logret_mean_std(tickers, data_map, by_column='Close') -> pd.DataFrame:
    '''Returns @result pd.DataFrame such that @result.loc[ticker] == [logret_mean, logret_std]'''
    result = pd.DataFrame(data=[], columns=['ticker', 'logret_mean', 'logret_std'])
    result.set_index('ticker', inplace=True)

    for ticker in tickers:
        ticker_data = data_map[ticker]
        ticker_logret = np.log(ticker_data[by_column]).diff()
        result.loc[ticker] = [ticker_logret.mean(), ticker_logret.std()]
    
    return result
  

for ticker in assets_data.keys():
    assets_data[ticker] = add_logret(assets_data[ticker])

estims = get_logret_mean_std(assets, assets_data)
estims

,logret_mean,logret_std
ticker,,
MDLZ,0.000164,0.009635
MSFT,0.001743,0.012638
NXPI,0.001406,0.023336
PCAR,-0.000078,0.015266
INTC,0.000118,0.020613
NVDA,0.003541,0.025897
ILMN,0.000500,0.020907
DXCM,0.002243,0.022329
ROST,0.000076,0.018766


#### Находим матрицу выборочных ковариаций доходностей 

In [32]:
def get_covariation_matrix(tickers,                          
                           data_map=data_for_ticker,
                           by_column='logret'):
    columns = pd.DataFrame()
    for ticker in tickers:
        columns[ticker] = data_map[ticker][by_column][1:]
        
    matrix = columns.corr()
    return matrix

cov_matrix = get_covariation_matrix(assets, data_map=assets_data)
cov_matrix

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
MDLZ,1.000000,0.260625,0.136202,0.166086,0.128461,0.071711,0.106817,0.121394,0.320069,0.200534,0.142022,0.184870,0.108808,0.183858,0.145688,0.074572,-0.016072,0.090393,0.076097,0.188254
MSFT,0.260625,1.000000,0.435936,0.129668,0.343282,0.563342,0.365982,0.396934,0.237147,0.369962,0.340569,0.520006,0.156858,0.460941,0.266493,0.374816,0.088876,-0.054785,0.171884,0.002140
NXPI,0.136202,0.435936,1.000000,0.314621,0.518692,0.567564,0.348801,0.352195,0.347588,0.422580,0.419310,0.465200,0.366023,0.232486,0.300793,0.576943,0.114702,0.194865,0.201715,0.371353
PCAR,0.166086,0.129668,0.314621,1.000000,0.285791,0.146135,0.077887,0.024713,0.310914,0.112148,0.185204,0.087999,0.327231,-0.011242,0.357433,0.209517,0.091692,0.189116,0.160347,0.414896
INTC,0.128461,0.343282,0.518692,0.285791,1.000000,0.352533,0.238309,0.220853,0.287966,0.255882,0.231893,0.245138,0.141588,0.236885,0.280478,0.366278,0.132862,0.064409,0.115456,0.216980
NVDA,0.071711,0.563342,0.567564,0.146135,0.352533,1.000000,0.296262,0.351007,0.154648,0.415916,0.425477,0.490961,0.093375,0.356414,0.243640,0.441977,0.087635,-0.070881,0.113711,-0.013629
ILMN,0.106817,0.365982,0.348801,0.077887,0.238309,0.296262,1.000000,0.433226,0.030674,0.326248,0.271251,0.369235,0.078505,0.249747,0.162399,0.236065,0.031524,0.025273,0.094362,0.011555
DXCM,0.121394,0.396934,0.352195,0.024713,0.220853,0.351007,0.433226,1.000000,0.161316,0.370966,0.193181,0.345706,0.067249,0.343953,0.184725,0.343466,-0.024543,0.041092,0.198570,0.015732
ROST,0.320069,0.237147,0.347588,0.310914,0.287966,0.154648,0.030674,0.161316,1.000000,0.287834,0.314949,0.218192,0.409139,0.082407,0.331344,0.318590,-0.018868,0.234797,0.131619,0.430977
LULU,0.200534,0.369962,0.422580,0.112148,0.255882,0.415916,0.326248,0.370966,0.287834,1.000000,0.315082,0.393846,0.156097,0.306344,0.189652,0.252620,-0.010867,0.026117,0.062358,0.081343


In [33]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Correlation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn,
                zmin=-1, zmax=1)
fig.show()

#### Проверяем вырожденность матрицы и число обусловленности

In [34]:
det = np.linalg.det(cov_matrix)
print(f'det = {det}')

con = np.linalg.cond(cov_matrix, p='fro')
print(f'con = {con}')

det = 0.002219356005352476
con = 55.688725974909254


### 1. Истинный оптимальный портфель в модели Марковица с заданным отношением к риску. 

С заданным отношением к риску  подобираем константу b таким образом, что истинный оптимальный CVaR портфель совпадает с истинным оптимальным портфелем п.1. Значение константы смотри в упражнениях к теме.

$$ b = \frac{1}{\sqrt{2 \pi }} \frac{1}{(1 - \beta)} exp(-(\Phi^{-1}(\beta))^2 / 2)$$

In [35]:
import math
from scipy.stats import norm

beta = 0.95
b = (1 / math.sqrt(2 * math.pi)) * (1 / (1 - beta)) * np.exp(-((norm.ppf(beta))**(-1))**2 / 2) 
print(f'b = {b}')

b = 6.632540979594675


#### Решаем задачу оптимизации 

$$ -E(x)+ с \cdot σ(x) \rightarrow min $$
при условиях:
$$ x_1 + x_2 + ... +x_N = 1 $$
$$ x_i >= 0 $$
где
$$ E(x)= E_1x_1 + E_2 x_2 + ... + E_Nx_N $$
$$ σ^2(x)=\Sigma\Sigma σ_{i,j} x_i \cdot x_j $$

In [36]:
import math 
import numpy as np
from scipy.optimize import minimize

def get_E(x, vector_E):
    E = sum([(E_i * x_i) for E_i, x_i in zip(vector_E, x)]) 
    return E


def get_sigma(x, matrix_cov):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = matrix_cov.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function(x, vector_E, matrix_cov, minusE):
    E = get_E(x, vector_E)
    sigma = get_sigma(x, matrix_cov)
    if minusE:
        result = -E + b * sigma
    else:
        result = E + b * sigma
    return result


def find_optimal(vector_E=estims['logret_mean'], matrix_cov=cov_matrix, minusE=True):
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(target_function, x0, args=(vector_E, matrix_cov, minusE), 
                        method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    
true_opt_solution = find_optimal()

#### Веса портфеля:

In [37]:
import plotly.express as px

print(f'Сумма весов: {sum(true_opt_solution.x)}')
fig = px.bar(x=assets, y=true_opt_solution.x)
fig.show()

Сумма весов: 1.0000000000000002


#### Значение целевой функции:

In [38]:
print(true_opt_solution.fun)

2.881357586257086


In [39]:
def get_portfolio_coordinates(weights,
                              _tickers,
                              _logret_mean,
                              _logret_std):
    portfolio_logret_mean = 0
    portfolio_logret_std = 0
     
    for i in range(len(_tickers)):
        portfolio_logret_mean += weights[i] * _logret_mean[i]
        for j in range(len(_tickers)):
            portfolio_logret_std += weights[i] * weights[j] * _logret_std.loc[_tickers[i], _tickers[j]]
            
    portfolio_logret_std = math.sqrt(portfolio_logret_std)
    return portfolio_logret_std, portfolio_logret_mean 

In [40]:
import plotly.express as px
from sklearn.preprocessing import normalize

ticker_colour = estims['logret_mean'] / estims['logret_std']
ticker_size = (ticker_colour - ticker_colour.min()) / ticker_colour.max()
fig = px.scatter(estims, x='logret_std', y='logret_mean',
                 hover_name=estims.index, 
                 color_continuous_scale=px.colors.diverging.RdYlGn,
                 color=ticker_colour,
                 size=ticker_size)
fig.show()

In [42]:
# _x, _y = get_portfolio_coordinates(true_opt_solution.x, assets, estims['logret_mean'], estims['logret_std'])

### 2. Оценка неопределенности оптимального портфеля в модели Марковица с заданным отн. к риску. 

#### 2.1 Задаём число наблюдений T=30. С помощью генератора многомерного нормального распределения создаём выборку размера Т из нормального распределения с вектором математических ожиданий  E=(E1, E2, …, EN) и матрицей ковариаций (σi,j). 

In [43]:
T = 30
sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
sample = pd.DataFrame(columns=assets, data=sample_raw)
sample

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
0,-2.653609,-1.616178,-1.681353,-3.035527,-0.130507,-1.196621,-1.365826,-0.108234,-1.821851,-1.144473,-1.261570,-3.407088,-1.653809,-1.509995,-2.228630,-2.247702,-0.778036,0.047298,-2.324188,-1.624171
1,0.174216,2.589333,1.299973,0.312618,0.279158,1.389699,1.284274,1.224546,-0.880193,2.030823,1.133514,1.692833,-0.269323,1.731466,-0.303781,0.980047,-0.161166,1.618292,-0.793027,0.152878
2,0.997779,-0.139380,0.507367,2.175557,-0.869742,1.295231,-1.018486,-0.775086,1.107556,-0.674963,0.412634,0.407439,-0.348432,0.919009,0.437695,0.672586,1.959113,0.145779,0.828908,0.762968
3,-0.366351,0.971911,-0.186879,-0.833073,0.394053,0.809659,0.793569,0.605797,-1.717243,-0.039861,0.072227,1.053433,-1.628755,0.695462,-0.340797,-0.114436,1.972278,-0.664554,-0.296832,-2.079341
4,0.166950,-0.358551,-0.374269,-0.558043,0.533280,-0.509099,-0.912766,-1.745180,1.929099,-0.040969,1.664470,1.177751,0.000792,0.091978,-1.053663,0.110086,0.681420,0.643330,-1.100437,0.200052
5,-0.935488,0.574212,-0.643267,1.156368,-0.294324,0.542658,0.228853,0.256633,1.210764,0.284389,1.397073,0.011705,0.729525,0.197700,1.499898,0.454789,-0.581429,1.773552,1.511196,-0.381495
6,0.810275,0.131027,-0.344631,0.728398,-1.742730,0.420086,-0.693281,0.444660,-0.401605,-0.745895,-0.140949,0.490530,0.185164,0.439476,-0.750112,-1.182284,-0.429196,-1.046803,-0.239065,-0.183698
7,0.991526,-0.587706,-0.793760,0.527985,0.245255,0.288077,-1.103051,-1.655520,-0.575819,0.106050,-0.221927,-0.013324,-0.343036,1.222635,0.641393,-1.160792,0.585178,0.958658,-2.698550,-1.109112
8,-2.078697,-1.450852,-0.625674,-0.792567,0.574373,-0.394631,-0.965813,0.326075,0.247023,0.364450,0.683845,-0.927438,0.290184,-1.777973,0.876418,-0.671934,-0.024244,-0.884977,0.739898,-0.240434
9,1.436080,-1.291734,-1.115941,0.743359,1.086179,0.025483,-0.493246,0.104818,0.822826,0.800432,0.306799,-1.612646,-0.970611,0.544967,0.761712,-0.303578,0.168402,-1.003091,-1.301612,-0.004660


#### 2.2 По построенной выборке делаем оценку Eest вектора математических ожиданий ...

In [44]:
estE = sample.mean()
estE

MDLZ    -0.046599
MSFT    -0.307708
NXPI    -0.269952
PCAR     0.093141
INTC     0.054453
NVDA    -0.027328
ILMN    -0.261669
DXCM    -0.205815
ROST    -0.077558
LULU     0.039106
PRFT     0.068137
FTNT    -0.213982
WIRE    -0.176481
NFLX    -0.141903
GAIN    -0.148621
SYNA    -0.081877
NYMTP    0.133744
ANAT    -0.102992
TRNS    -0.325066
SFBS     0.168044
dtype: float64

#### ... и оценку (σesti,j) матрицы ковариаций. 

In [45]:
estCov = sample.corr()
estCov

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
MDLZ,1.000000,0.332384,0.155499,0.399503,0.196195,0.119303,0.094587,-0.117447,0.292761,0.031796,0.085872,0.257111,-0.138276,0.638379,0.405451,0.181472,0.256430,0.035275,-0.079052,0.251109
MSFT,0.332384,1.000000,0.562540,0.196024,0.165362,0.499580,0.452237,0.366873,0.357469,0.246098,0.465644,0.721590,0.194157,0.346385,0.555899,0.527409,0.359684,0.326155,0.060474,0.282149
NXPI,0.155499,0.562540,1.000000,0.415676,0.440604,0.455370,0.344767,0.085658,0.398864,0.189192,0.279843,0.552453,0.247190,0.264029,0.475239,0.559660,0.475687,0.371636,0.240855,0.474538
PCAR,0.399503,0.196024,0.415676,1.000000,0.082323,0.046250,-0.007664,-0.068560,0.531677,0.121144,0.120308,0.249825,0.480513,0.237011,0.449049,0.321981,0.257725,0.539967,0.334286,0.498480
INTC,0.196195,0.165362,0.440604,0.082323,1.000000,0.139535,0.271066,-0.086145,0.361728,0.245222,0.275830,0.179725,-0.169248,0.381392,0.524314,0.251816,0.353219,0.198897,-0.106101,0.148714
NVDA,0.119303,0.499580,0.455370,0.046250,0.139535,1.000000,0.317448,0.313561,0.111310,0.462473,0.562576,0.590204,0.192946,0.418569,0.441944,0.602903,0.492891,-0.016077,0.075607,-0.154834
ILMN,0.094587,0.452237,0.344767,-0.007664,0.271066,0.317448,1.000000,0.523719,0.066988,0.428690,0.547770,0.303888,0.151743,0.279399,0.310666,0.297487,0.188028,0.044653,-0.017183,0.152792
DXCM,-0.117447,0.366873,0.085658,-0.068560,-0.086145,0.313561,0.523719,1.000000,-0.126243,0.429088,0.250071,0.115268,0.217283,-0.063880,0.142217,0.152413,-0.076571,-0.160448,0.112125,-0.025834
ROST,0.292761,0.357469,0.398864,0.531677,0.361728,0.111310,0.066988,-0.126243,1.000000,0.123199,0.467424,0.382001,0.481181,0.132563,0.713155,0.475907,0.277335,0.429651,0.135869,0.634947
LULU,0.031796,0.246098,0.189192,0.121144,0.245222,0.462473,0.428690,0.429088,0.123199,1.000000,0.511468,0.416550,0.173578,0.386859,0.307674,0.284249,0.043090,0.045413,-0.221852,0.085310


In [46]:
import plotly.express as px

fig = px.imshow(estCov, title='Correlation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn, 
                zmin=-1, zmax=1)
fig.show()

#### 2.3 Используя эти оценки решаем задачу оптимизации 

In [47]:
opt_solution = find_optimal(estE, estCov)

#### Находим и фиксируем веса портфеля ...

In [48]:
print(f'Сумма весов: {sum(opt_solution.x)}')
fig = px.bar(x=assets, y=opt_solution.x)
fig.show()

Сумма весов: 1.0000000000000029


#### и значение целевой функции

In [49]:
print(opt_solution.fun)

2.918238358413867


#### 2.4 Сравниаем два портфеля: истинный (п.1) и выборочный (п.2.3). Оцениваем относительную ошибку в определении весов портфеля в норме Manhattan (L1 норма Минковского). Делайте выводы и сравнение в системе координат (σ, E). 

In [50]:
from scipy.spatial.distance import cityblock
cityblock(true_opt_solution.x, opt_solution.x)

0.8747235193645739

#### 2.5 Повторите эксперимент S=40 раз и оцените среднюю относительную ошибку по S повторениям эксперимента. Сделайте выводы.  Сделайте сравнение в системе координат (σ, E). 

In [51]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estE, estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

KeyboardInterrupt: 

In [ ]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()

#### 2.6  Предположите, что нам известны точные значения математических ожиданий E=(E1, E2, …, EN). Повторяем пп. 2.2-2.5. используя оценку только матрицы ковариаций

In [ ]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estims['logret_mean'], estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

In [ ]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()

### 3. Оценка неопределенности оптимального CVaR портфеля

С заданным отношением к риску  подобираем константу b таким образом, что истинный оптимальный CVaR портфель совпадает с истинным оптимальным портфелем п.1. Значение константы смотри в упражнениях к теме.

$$ b = \frac{1}{\sqrt{2 \pi }} \frac{1}{(1 - \beta)} exp(-(\Phi^{-1}(\beta))^2 / 2)$$

In [ ]:
beta = 0.95
b = (1 / math.sqrt(2 * math.pi)) * (1 / (1 - beta)) * np.exp(-((norm.ppf(beta))**(-1))**2 / 2) 
print(f'b = {b}')

In [ ]:
def get_E(x, vector_E):
    E = sum([(E_i * x_i) for E_i, x_i in zip(vector_E, x)]) 
    return E


def get_sigma(x, matrix_cov):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = matrix_cov.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function_CVaR(x, vector_E, matrix_cov):
    E = get_E(x, vector_E)
    sigma = get_sigma(x, matrix_cov)
    result = E + b * sigma
    return result


def find_optimal_CVaR(vector_E, matrix_cov):
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(target_function_CVaR, x0, args=(vector_E, matrix_cov), 
                        method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    

#### Используя оценки из 2.2 решаем задачу оптимизации 

In [ ]:
opt_solution_CVaR = find_optimal_CVaR(estE, estCov)

#### Находим и фиксируем веса портфеля ...

In [ ]:
print(f'Сумма весов: {sum(opt_solution_CVaR.x)}')
fig = px.bar(x=assets, y=opt_solution_CVaR.x)
fig.show()

#### и значение целевой функции

In [ ]:
print(opt_solution_CVaR.fun)

#### 3.2 Сравниаем два портфеля: истинный (п.1) и найденный в п.3.1. Оцениваем относительную ошибку в определении весов портфеля в норме Manhattan (L1 норма Минковского). Сравниваем с ошибкой портфеля из п. 2.3

In [ ]:
er1 = cityblock(true_opt_solution.x, opt_solution.x)
er2 = cityblock(true_opt_solution.x, opt_solution_CVaR.x)
print(f'Ошибка оптимального VaR портфеля(п.2.3) = {er1}')
print(f'Ошибка оптимального CVaR портфеля = {er2}')

#### 3.3 Повторите эксперимент S=40 раз и оцените среднюю относительную ошибку по S повторениям эксперимента. Сделайте выводы.  Сравните с ошибкой из п. 2.5.

In [ ]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution_CVaR = find_optimal_CVaR(estE, estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution_CVaR.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution_CVaR.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

In [ ]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()